In [1]:
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import numpy as np
import pandas as pd
import pickle

In [2]:
def GetTodayOption(year, month, day):
    keys = {'syear': year, 'smonth': month, 'sday': day} # 要傳給期貨交易所的 key
    r = requests.get("http://www.taifex.com.tw/chinese/3/3_2_1.asp", data = keys) # 抓取這個 key 回傳網頁的原始碼
    r.encoding='utf8'
    soup = BeautifulSoup(r.text , "html.parser") # 把原始碼做整理
    soup_data1 = pd.read_html(str(soup.select('table')[2]),header=0)[2] ## 我不知道為什麼最後是[2]
    soup_data2 = pd.read_html(str(soup.select('table')[5]),header=0)[2]
    soup_data1=soup_data1.drop(['最高','最低','結算價','成交量','未平倉','漲跌'],axis=1)
    soup_data2=soup_data2.drop(['到期月份(週期)','最高','最低','結算價','成交量','未平倉','漲跌','履約價'],axis=1)
    df = pd.concat([soup_data1,soup_data2],axis=1).replace('-','NaN').convert_objects(convert_numeric=True)
    df.columns = ['到期月份','履約價','Call_最後成交價','Put_最後成交價']
    
    
    df['需刪除履約價'] = pd.DataFrame(df['履約價']%100)
    df['compare'] =df['Call_最後成交價'] - df['Put_最後成交價']
    
    df = df[df.需刪除履約價 != 50]
    
    df = pd.concat([df['到期月份'],df['履約價'],df['Call_最後成交價'],df['Put_最後成交價'],df['compare']],axis=1)
    df = df.reset_index(drop=True)
    
    return df

In [3]:
def  third_wen(year_2,month_2):                         # 此函數需參數 年 及 月
    day_2=21-(dt.date(year_2,month_2,1).weekday()+4)%7    # weekday函數 禮拜一為0;禮拜日為6
    return year_2,month_2,day_2

d = {}
for i in range(2010,2018):
    for j in range(1,13):
        d[i,j] = third_wen(i,j)
res = pickle.dumps(d)

A = pd.DataFrame.from_dict(d,orient='index')#.convert_objects(convert_numeric=True)
A.columns = ['年','月','日']
date = pd.concat([A['年'],A['月'],A['日']],axis=1)
year_2 = list(A['年'])
month_2 = list(A['月'])
day_2 =list(A['日'])

In [10]:
df_2 = pd.DataFrame()

for i in range(len(year_2)):
    timeup=GetTodayOption(year_2[0+i],month_2[0+i],day_2[0+i])
    #temp=temp.dropna()
    #temp = temp[np.isfinite(temp['到期月份'])]
    print(i)
    
    try:
        date_2=np.isnan(timeup['到期月份'])
        date_2=list(date_2)
        loc=date_2.index(0)
        loc
        date_2=timeup['到期月份'][loc]
        index=timeup['到期月份']==date_2
        timeup=timeup[index]    
        timeup=timeup.reset_index(drop=True)
            #抓出價平
            
            
            
        Cmp_2 = timeup['compare']
        Cmp_2 = abs(Cmp_2)
        Cmp_2.min(skipna=True)
        loc = abs(Cmp_2)==Cmp_2.min(skipna=True)
        loc=list(loc)
        loc=loc.index(1)
        
        
        Cmp_2 = timeup[(loc):(loc+6)]
        

        
        
        
        df_2=pd.concat([df_2,Cmp_2])
        
    except Exception as e:
        print ('stock market was not open on %d/%d/%d' %(year_2[0+i], month_2[0+i], day_2[0+i]))
        pass
    

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


0
1
stock market was not open on 2010/2/17
2
3
4
5
stock market was not open on 2010/6/16
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
stock market was not open on 2013/8/21
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
stock market was not open on 2015/2/18
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [11]:
df

,到期月份,履約價,Call_最後成交價,Put_最後成交價,compare
5,201001.0,7800,415.0,0.1,414.9
6,201001.0,7900,317.0,0.2,316.8
7,201001.0,8000,215.0,0.1,214.9
8,201001.0,8100,114.0,0.1,113.9
9,201001.0,8200,14.5,0.1,14.4
10,201001.0,8300,0.1,85.0,-84.9
11,201001.0,8400,0.2,185.0,-184.8
12,201001.0,8500,0.2,286.0,-285.8
13,201001.0,8600,0.1,385.0,-384.9
5,201003.0,7400,443.0,0.1,442.9


In [12]:
df_2=df_2.replace(201102.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201204.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201305.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201401.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201702.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201710.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
C

,到期月份,履約價,Call_最後成交價,Put_最後成交價,compare
9,201001.0,8200,14.5,0.1,14.4
10,201001.0,8300,0.1,85.0,-84.9
11,201001.0,8400,0.2,185.0,-184.8
12,201001.0,8500,0.2,286.0,-285.8
13,201001.0,8600,0.1,385.0,-384.9
14,201001.0,8700,0.2,510.0,-509.8
9,201003.0,7800,43.0,0.2,42.8
10,201003.0,7900,0.3,57.0,-56.7
11,201003.0,8000,0.2,156.0,-155.8
12,201003.0,8100,0.2,257.0,-256.8


In [13]:
df.to_csv('timeup.csv')